## 0. Objetivo

### Analisis del mismo modelo sobre los mismos datos pero con distintas estrategias de compilacion del modelo con la libreria Tensorflow


## Arquitectura, version Cuda...

In [1]:
!nvidia-smi

Fri Jul 25 20:26:03 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 576.88                 Driver Version: 576.88         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 960       WDDM  |   00000000:01:00.0 Off |                  N/A |
| 24%   40C    P8             19W /  160W |    1542MiB /   2048MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Seguidamente establecemos una strategia global de espejp con la gpu y una reserva que haremos en la ram

In [3]:
import tensorflow as tf

tf.debugging.set_log_device_placement(False)
gpus = tf.config.list_logical_devices('GPU')
strategy0 = tf.distribute.MirroredStrategy(gpus)
strategy1 = tf.distribute.MirroredStrategy(cross_device_ops=tf.distribute.NcclAllReduce())
strategy2 = tf.distribute.MirroredStrategy(devices=gpus.append([tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)]), cross_device_ops=tf.distribute.ReductionToOneDevice(), )

tf.config.list_physical_devices('GPU')
tf.test.is_built_with_cuda()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


False

Comprobamos

In [4]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)


1 Physical GPUs, 1 Logical GPU


In [5]:
# Importar bibliotecas necesarias

import os
from openpyxl import load_workbook

import numpy as np  # Para trabajar con arreglos numéricos y generar datos sintéticos
import matplotlib.pyplot as plt  # Para graficar resultados
import seaborn as sns  # Para crear gráficos más atractivos
from sklearn.metrics import confusion_matrix  # Para analizar errores con una matriz de confusión
from sklearn.model_selection import train_test_split  # Para dividir los datos en entrenamiento y validación

import pandas as pd
# Estas son herramientas para construir, entrenar y evaluar redes neuronales.
import keras as keras  # Biblioteca para trabajar con redes neuronales
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense, InputLayer, Dropout, BatchNormalization, LeakyReLU
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.regularizers import l2



#------------------------------------------------
import sklearn

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler  # Para escalar los datos
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import random
from time import time

In [6]:

# Fijar semillas para reproducibilidad
SEED = 5785630 #random.randint(0,100000)	  # Generar un número aleatorio entre 0 y 1000
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)
sklearn.random.seed(SEED)
print(f"Seed: {SEED}")  # Imprimir la semilla utilizada

Seed: 5785630


In [7]:
def reparar_nombres(file):
    import os

    if " " in file:
        os.rename(file, file.replace(" ", "_"))
    return file.replace(" ", "_")


scores = {}

In [8]:
# pillar los xls de la carpeta y los mete en un dataframe conjunto

path = "./"
archivos = [x for x in os.listdir(path) if x.endswith(".xlsx")]
df = pd.DataFrame()
df_cols = ""
for archivo in archivos:
    archivo = reparar_nombres(archivo)

    archivo = path + str(archivo)

    xlsx = load_workbook(archivo)

    paginas = xlsx.sheetnames

    dfx = pd.read_excel(archivo, sheet_name=paginas[0])

    dfy = pd.read_excel(archivo, sheet_name=paginas[1])


    df_tmp = pd.concat([dfx, dfy], axis=1)
    if len(df.columns) != 0:
        df.columns = df_tmp.columns
    df = pd.concat([df, df_tmp], axis=0, ignore_index=True)

df = df.dropna(axis=0, how="any")
df = df.drop_duplicates()
df.to_csv("df.csv", index=False)


In [9]:
df = pd.read_csv("df.csv")

In [10]:
dfx = df.iloc[:, :-8]
dfy = df.iloc[:, -8:]

In [11]:
import re

for i in dfy.columns:
    dfy[i] = dfy[i].apply(lambda x: x if type(x) is not str else float(re.sub(r'[^0-9.]', '', x)))

for i in dfx.columns:
    dfx[i] = dfx[i].apply(lambda x: x if type(x) is not str else float(re.sub(r'[^0-9.]', '', x)))

dfx = dfx.astype(float)
dfy = dfy.astype(float)

In [12]:
dfx.describe()

,380.0,380.5,381.0,381.5,382.0,382.5,383.0,383.5,384.0,384.5,...,775.0,775.5,776.0,776.5,777.0,777.5,778.0,778.5,779.0,779.5
count,671.000000,671.000000,671.000000,671.000000,671.000000,671.000000,671.000000,671.000000,671.000000,671.000000,...,671.000000,671.0,671.0,671.000000,671.0,671.0,671.0,671.0,671.0,671.0
mean,23.548903,27.147522,24.603996,24.852823,26.199975,27.165705,32.306966,30.009750,35.036629,32.437839,...,0.968703,0.0,0.0,0.149031,0.0,0.0,0.0,0.0,0.0,0.0
std,37.342702,46.220070,37.468848,36.315064,39.065509,43.198508,50.323903,45.720979,59.114661,45.700615,...,8.363955,0.0,0.0,3.860457,0.0,0.0,0.0,0.0,0.0,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
75%,47.722500,50.000000,50.757500,50.000000,51.257000,54.919500,62.243000,62.397500,61.064000,63.599500,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
max,300.000000,500.000000,200.000000,300.000000,206.250000,400.000000,400.000000,500.000000,500.000000,400.000000,...,100.000000,0.0,0.0,100.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
dfy.describe()


,Grado Alcohólico Adquirido (%Vol),Ácidez Total Tartárica (g/L),pH,Ácidez Volátil Acética (g/L),Anhídrido Sulfuroso libre (mg/L),Anhídrido Sulfuroso Total (mg/L),Azúcares reductores (g/L),Ácido Málico (g/L)
count,671.000000,671.000000,671.000000,671.000000,671.000000,671.000000,671.000000,671.000000
mean,12.735395,7.010462,3.180909,0.393472,10.306706,67.608048,1.970238,2.043070
std,0.528579,1.074549,0.090193,0.213059,7.684913,44.516237,3.922582,1.181123
min,11.120000,4.600000,3.040000,0.180000,5.000000,5.000000,0.170000,0.100000
25%,12.670000,6.370000,3.130000,0.300000,5.000000,36.000000,1.000000,1.300000
50%,12.880000,6.800000,3.160000,0.340000,5.000000,63.000000,1.400000,1.900000
75%,13.070000,7.560000,3.210000,0.410000,16.000000,85.000000,2.000000,2.800000
max,13.590000,9.610000,3.510000,1.560000,39.000000,192.000000,35.000000,4.400000


In [14]:

N_Datos = dfx.shape[0]  # Se generarán 81 ejemplos o muestras.
N_Dimensiones_entrada_X = dfx.shape[1]  # Cada muestra tendrá 800 características (dimensiones de entrada).
N_Dimensiones_salida_Y = dfy.shape[1]  # Habrá 8 clases posibles para la salida.

X = pd.DataFrame(dfx)  # Genera datos de entrada (X) de forma aleatoria con valores flotantes entre 0 y 1.
y = pd.DataFrame(dfy)  # Genera etiquetas de salida (Y) de forma aleatoria con valores enteros entre 0 y 9.


In [15]:


# --- Preprocesamiento ---
scaler_x = StandardScaler()
X_scaled = scaler_x.fit_transform(X)
scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y)

X_train, X_test, Y_train_scaled, Y_test_scaled = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=SEED)



In [16]:
["mse" for _ in range(0, 8)]

['mse', 'mse', 'mse', 'mse', 'mse', 'mse', 'mse', 'mse']

In [17]:
import tensorflow as tf
print("Versión de TensorFlow:", tf.__version__)
print("GPUs detectadas:", tf.config.list_physical_devices('GPU'))


Versión de TensorFlow: 2.10.0
GPUs detectadas: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## GPU0

In [18]:

# --- Red neuronal regresora multisalida ---
start_time = time()
early = EarlyStopping(monitor='loss', patience=25, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='loss', patience=15, factor=0.5)
with tf.device('/device:GPU:0'):
    # Definir el modelo
    model0 = Sequential(
        [
            InputLayer(input_shape=(N_Dimensiones_entrada_X,)),  # Capa de entrada con 8000 características
            BatchNormalization(),  # Normalización por lotes para mejorar la convergencia
            LeakyReLU(0.5),
            Dense(512, activation="leaky_relu", kernel_regularizer=l2(0.001)),  # Capa oculta con 1024 neuronas
            Dropout(0.2),  # Capa de abandono para evitar el sobreajuste
            LeakyReLU(0.25),
            Dense(128, activation="leaky_relu"),  # Otra capa oculta con 512 neuronas
            Dropout(0.1),  # Capa de abandono para evitar el sobreajuste
            Dense(16, activation="relu", kernel_regularizer=l2(0.001)),  # Otra capa oculta con 256 neuronas
            Dense(N_Dimensiones_salida_Y),  # Capa de salida con 8 valores (Target)
        ]
    )

    adam = Adam(learning_rate=0.001)
    # Compilar el modelo
    model0.compile(optimizer=adam , loss=["mse", "mse", "mse", "mse", "mse", "mse", "mse", "mse"], loss_weights=(0.9,0.4,0.3,0.4,0.8,0.1,0.3,0.7),
                  metrics=["mae","mse"])  # Función de pérdida MSE y métricas MAE y MSE
    
end_time = time()
print(f"Tiempo de compilacion del modelo 0: {end_time - start_time:.2f} segundos")
# Resumen del modelo

Tiempo de compilacion del modelo 0: 0.37 segundos


In [19]:

model0.summary()
start_time = time()
# --- Entrenamiento ---
history0 = model0.fit(X_train, Y_train_scaled, epochs=500, batch_size=32, validation_data=(X_test, Y_test_scaled), verbose=1, callbacks=[early, reduce_lr])
stop_time = time()
print(f"Tiempo de entrenamiento del modelo 0: {stop_time - start_time:.2f} segundos")


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 800)              3200      
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 800)               0         
                                                                 
 dense (Dense)               (None, 512)               410112    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               65664     
                                                        

In [20]:


Y_pred_scaled0 = model0.predict(X_test)

# --- Evaluación ---
Y_pred0 = scaler_y.inverse_transform(Y_pred_scaled0)
Y_test0 = scaler_y.inverse_transform(Y_test_scaled)
aux = {}
aux['r2'] = []
aux['mae'] = []
aux['mse'] = []
aux['rmse'] = []


""" # --- Evaluación ---
# Errores por cada salida
for i in range(N_Dimensiones_salida_Y):
    print(f"--- Output {i+1} ---")
    print("  R2 :", r2_score(Y_test0[:, i], Y_pred0[:, i]))
    aux['r2'].append(r2_score(Y_test0[:, i], Y_pred0[:, i]))
    print("MAE:", mean_absolute_error(Y_test0[:, i], Y_pred0[:, i]))
    aux['mae'].append(mean_absolute_error(Y_test0[:, i], Y_pred0[:, i]))
    print("  MSE:", mean_squared_error(Y_test0[:, i], Y_pred0[:, i]))
    aux['mse'].append(mean_squared_error(Y_test0[:, i], Y_pred0[:, i]))
    print("  RMSE:", np.sqrt(mean_squared_error(Y_test0[:, i], Y_pred0[:, i])))
    aux['rmse'].append(np.sqrt(mean_squared_error(Y_test0[:, i], Y_pred0[:, i])))
aux = pd.DataFrame(aux)

import matplotlib.pyplot as plt

for i in range(N_Dimensiones_salida_Y):
    plt.figure(figsize=(5,4))
    plt.scatter(Y_test0[:, i], Y_pred0[:, i], alpha=0.5)
    plt.plot([Y_test0[:, i].min(), Y_test0[:, i].max()],
             [Y_test0[:, i].min(), Y_test0[:, i].max()],
             'r--')
    plt.xlabel("Valor real")
    plt.ylabel("Predicción")
    plt.title(f"{dfy.keys()[i]} - Pred vs Real")
    plt.grid()
    plt.tight_layout()
    plt.show()
     """
mae_total0 = mean_absolute_error(Y_test0.flatten(), Y_pred0.flatten())
mse_total0 = mean_squared_error(Y_test0.flatten(), Y_pred0.flatten())
r2_total0 = r2_score(Y_test0, Y_pred0)

print("\n--- Total ---")
print("R² :", r2_total0)
print("MAE:", mae_total0)
print("MSE:", mse_total0)

5/5 [==============================] - 0s 4ms/step

--- Total ---
R² : 0.8491401544808914
MAE: 1.9881885551025191
MSE: 39.125808040506755


## tf.distribute.MirroredStrategy(gpus)

In [21]:

# --- Red neuronal regresora multisalida ---
start_time = time()
early = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='loss', patience=5, factor=0.5)
with strategy0.scope():
    # Definir el modelo
    model1 = Sequential(
        [
            InputLayer(input_shape=(N_Dimensiones_entrada_X,)),  # Capa de entrada con 8000 características
            BatchNormalization(),  # Normalización por lotes para mejorar la convergencia
            LeakyReLU(0.5),
            Dense(512, activation="leaky_relu", kernel_regularizer=l2(0.001)),  # Capa oculta con 1024 neuronas
            Dropout(0.2),  # Capa de abandono para evitar el sobreajuste
            LeakyReLU(0.25),
            Dense(128, activation="leaky_relu"),  # Otra capa oculta con 512 neuronas
            Dropout(0.1),  # Capa de abandono para evitar el sobreajuste
            Dense(16, activation="relu", kernel_regularizer=l2(0.001)),  # Otra capa oculta con 256 neuronas
            Dense(N_Dimensiones_salida_Y),  # Capa de salida con 8 valores (Target)
        ]
    )

    adam = Adam(learning_rate=0.001)
    # Compilar el modelo
    model1.compile(optimizer=adam , loss=["mse", "mse", "mse", "mse", "mse", "mse", "mse", "mse"], loss_weights=(0.1,1,1,1,1,1,1,1), metrics=["mae","mse"])  # Función de pérdida MSE y métricas MAE y MSE
end_time = time()
print(f"Tiempo de compilacion del modelo 1: {end_time - start_time:.2f} segundos")

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [22]:

# Resumen del modelo
model1.summary()
start_time = time()
# --- Entrenamiento ---
history1 = model1.fit(X_train, Y_train_scaled, epochs=500, batch_size=32, validation_data=(X_test, Y_test_scaled), verbose=1, callbacks=[early, reduce_lr])
stop_time = time()
print(f"Tiempo de entrenamiento del modelo 1: {stop_time - start_time:.2f} segundos")

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_1 (Batc  (None, 800)              3200      
 hNormalization)                                                 
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 800)               0         
                                                                 
 dense_4 (Dense)             (None, 512)               410112    
                                                                 
 dropout_2 (Dropout)         (None, 512)               0         
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 512)               0         
                                                                 
 dense_5 (Dense)             (None, 128)               65664     
                                                      

In [23]:




Y_pred_scaled1 = model1.predict(X_test)

# --- Evaluación ---
Y_pred1 = scaler_y.inverse_transform(Y_pred_scaled1)
Y_test1 = scaler_y.inverse_transform(Y_test_scaled)
aux = {}
aux['r2'] = []
aux['mae'] = []
aux['mse'] = []
aux['rmse'] = []


""" # --- Evaluación ---
# Errores por cada salida
for i in range(N_Dimensiones_salida_Y):
    print(f"--- Output {i+1} ---")
    print("  R2 :", r2_score(Y_test1[:, i], Y_pred1[:, i]))
    aux['r2'].append(r2_score(Y_test1[:, i], Y_pred1[:, i]))
    print("MAE:", mean_absolute_error(Y_test1[:, i], Y_pred1[:, i]))
    aux['mae'].append(mean_absolute_error(Y_test1[:, i], Y_pred1[:, i]))
    print("  MSE:", mean_squared_error(Y_test1[:, i], Y_pred1[:, i]))
    aux['mse'].append(mean_squared_error(Y_test1[:, i], Y_pred1[:, i]))
    print("  RMSE:", np.sqrt(mean_squared_error(Y_test1[:, i], Y_pred1[:, i])))
    aux['rmse'].append(np.sqrt(mean_squared_error(Y_test1[:, i], Y_pred1[:, i])))
aux = pd.DataFrame(aux)

import matplotlib.pyplot as plt

for i in range(N_Dimensiones_salida_Y):
    plt.figure(figsize=(5,4))
    plt.scatter(Y_test1[:, i], Y_pred1[:, i], alpha=0.5)
    plt.plot([Y_test1[:, i].min(), Y_test1[:, i].max()],
             [Y_test1[:, i].min(), Y_test1[:, i].max()],
             'r--')
    plt.xlabel("Valor real")
    plt.ylabel("Predicción")
    plt.title(f"{dfy.keys()[i]} - Pred vs Real")
    plt.grid()
    plt.tight_layout()
    plt.show()
     """
mae_total1 = mean_absolute_error(Y_test1.flatten(), Y_pred1.flatten())
mse_total1 = mean_squared_error(Y_test1.flatten(), Y_pred1.flatten())
r2_total1 = r2_score(Y_test1, Y_pred1)

print("\n--- Total ---")
print("R² :", r2_total1)
print("MAE:", mae_total1)
print("MSE:", mse_total1)




5/5 [==============================] - 1s 5ms/step

--- Total ---
R² : 0.543577508715474
MAE: 3.835609642141119
MSE: 132.98521404547168


## tf.distribute.MirroredStrategy(cross_device_ops=tf.distribute.NcclAllReduce())

In [24]:

# --- Red neuronal regresora multisalida ---
start_time = time()
early = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='loss', patience=5, factor=0.5)
with strategy1.scope():
    # Definir el modelo
    model1 = Sequential(
        [
            InputLayer(input_shape=(N_Dimensiones_entrada_X,)),  # Capa de entrada con 8000 características
            BatchNormalization(),  # Normalización por lotes para mejorar la convergencia
            LeakyReLU(0.5),
            Dense(512, activation="leaky_relu", kernel_regularizer=l2(0.001)),  # Capa oculta con 1024 neuronas
            Dropout(0.2),  # Capa de abandono para evitar el sobreajuste
            LeakyReLU(0.25),
            Dense(128, activation="leaky_relu"),  # Otra capa oculta con 512 neuronas
            Dropout(0.1),  # Capa de abandono para evitar el sobreajuste
            Dense(16, activation="relu", kernel_regularizer=l2(0.001)),  # Otra capa oculta con 256 neuronas
            Dense(N_Dimensiones_salida_Y),  # Capa de salida con 8 valores (Target)
        ]
    )

    adam = Adam(learning_rate=0.001)
    # Compilar el modelo
    model1.compile(optimizer=adam , loss=["mse", "mse", "mse", "mse", "mse", "mse", "mse", "mse"], loss_weights=(0.1,1,1,1,1,1,1,1), metrics=["mae","mse"])  # Función de pérdida MSE y métricas MAE y MSE
end_time = time()
print(f"Tiempo de compilacion del modelo 1: {end_time - start_time:.2f} segundos")

Tiempo de compilacion del modelo 1: 0.18 segundos


In [25]:

# Resumen del modelo
model1.summary()
start_time = time()
# --- Entrenamiento ---
history1 = model1.fit(X_train, Y_train_scaled, epochs=500, batch_size=32, validation_data=(X_test, Y_test_scaled), verbose=1, callbacks=[early, reduce_lr])
stop_time = time()
print(f"Tiempo de entrenamiento del modelo 1: {stop_time - start_time:.2f} segundos")

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_2 (Batc  (None, 800)              3200      
 hNormalization)                                                 
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 800)               0         
                                                                 
 dense_8 (Dense)             (None, 512)               410112    
                                                                 
 dropout_4 (Dropout)         (None, 512)               0         
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 512)               0         
                                                                 
 dense_9 (Dense)             (None, 128)               65664     
                                                      

In [26]:




Y_pred_scaled1 = model1.predict(X_test)

# --- Evaluación ---
Y_pred1 = scaler_y.inverse_transform(Y_pred_scaled1)
Y_test1 = scaler_y.inverse_transform(Y_test_scaled)
aux = {}
aux['r2'] = []
aux['mae'] = []
aux['mse'] = []
aux['rmse'] = []


""" # --- Evaluación ---
# Errores por cada salida
for i in range(N_Dimensiones_salida_Y):
    print(f"--- Output {i+1} ---")
    print("  R2 :", r2_score(Y_test1[:, i], Y_pred1[:, i]))
    aux['r2'].append(r2_score(Y_test1[:, i], Y_pred1[:, i]))
    print("MAE:", mean_absolute_error(Y_test1[:, i], Y_pred1[:, i]))
    aux['mae'].append(mean_absolute_error(Y_test1[:, i], Y_pred1[:, i]))
    print("  MSE:", mean_squared_error(Y_test1[:, i], Y_pred1[:, i]))
    aux['mse'].append(mean_squared_error(Y_test1[:, i], Y_pred1[:, i]))
    print("  RMSE:", np.sqrt(mean_squared_error(Y_test1[:, i], Y_pred1[:, i])))
    aux['rmse'].append(np.sqrt(mean_squared_error(Y_test1[:, i], Y_pred1[:, i])))
aux = pd.DataFrame(aux)

import matplotlib.pyplot as plt

for i in range(N_Dimensiones_salida_Y):
    plt.figure(figsize=(5,4))
    plt.scatter(Y_test1[:, i], Y_pred1[:, i], alpha=0.5)
    plt.plot([Y_test1[:, i].min(), Y_test1[:, i].max()],
             [Y_test1[:, i].min(), Y_test1[:, i].max()],
             'r--')
    plt.xlabel("Valor real")
    plt.ylabel("Predicción")
    plt.title(f"{dfy.keys()[i]} - Pred vs Real")
    plt.grid()
    plt.tight_layout()
    plt.show() """
    
mae_total1 = mean_absolute_error(Y_test1.flatten(), Y_pred1.flatten())
mse_total1 = mean_squared_error(Y_test1.flatten(), Y_pred1.flatten())
r2_total1 = r2_score(Y_test1, Y_pred1)

print("\n--- Total ---")
print("R² :", r2_total1)
print("MAE:", mae_total1)
print("MSE:", mse_total1)




5/5 [==============================] - 1s 5ms/step

--- Total ---
R² : 0.6364423244613842
MAE: 3.50080744793404
MSE: 100.17318336197567


## tf.distribute.MirroredStrategy(devices=gpus.append([tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)]), cross_device_ops=tf.distribute.ReductionToOneDevice(), )

In [27]:

# --- Red neuronal regresora multisalida ---
start_time = time()
early = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='loss', patience=5, factor=0.5)
with strategy2.scope():
    # Definir el modelo
    model1 = Sequential(
        [
            InputLayer(input_shape=(N_Dimensiones_entrada_X,)),  # Capa de entrada con 8000 características
            BatchNormalization(),  # Normalización por lotes para mejorar la convergencia
            LeakyReLU(0.5),
            Dense(512, activation="leaky_relu", kernel_regularizer=l2(0.001)),  # Capa oculta con 1024 neuronas
            Dropout(0.2),  # Capa de abandono para evitar el sobreajuste
            LeakyReLU(0.25),
            Dense(128, activation="leaky_relu"),  # Otra capa oculta con 512 neuronas
            Dropout(0.1),  # Capa de abandono para evitar el sobreajuste
            Dense(16, activation="relu", kernel_regularizer=l2(0.001)),  # Otra capa oculta con 256 neuronas
            Dense(N_Dimensiones_salida_Y),  # Capa de salida con 8 valores (Target)
        ]
    )

    adam = Adam(learning_rate=0.001)
    # Compilar el modelo
    model1.compile(optimizer=adam , loss=["mse", "mse", "mse", "mse", "mse", "mse", "mse", "mse"], loss_weights=(0.1,1,1,1,1,1,1,1), metrics=["mae","mse"])  # Función de pérdida MSE y métricas MAE y MSE
end_time = time()
print(f"Tiempo de compilacion del modelo 1: {end_time - start_time:.2f} segundos")

Tiempo de compilacion del modelo 1: 0.26 segundos


In [28]:

# Resumen del modelo
model1.summary()
start_time = time()
# --- Entrenamiento ---
history1 = model1.fit(X_train, Y_train_scaled, epochs=500, batch_size=32, validation_data=(X_test, Y_test_scaled), verbose=1, callbacks=[early, reduce_lr])
stop_time = time()
print(f"Tiempo de entrenamiento del modelo 1: {stop_time - start_time:.2f} segundos")

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_3 (Batc  (None, 800)              3200      
 hNormalization)                                                 
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 800)               0         
                                                                 
 dense_12 (Dense)            (None, 512)               410112    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 leaky_re_lu_7 (LeakyReLU)   (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 128)               65664     
                                                      

In [29]:




Y_pred_scaled1 = model1.predict(X_test)

# --- Evaluación ---
Y_pred1 = scaler_y.inverse_transform(Y_pred_scaled1)
Y_test1 = scaler_y.inverse_transform(Y_test_scaled)
aux = {}
aux['r2'] = []
aux['mae'] = []
aux['mse'] = []
aux['rmse'] = []


""" # --- Evaluación ---
# Errores por cada salida
for i in range(N_Dimensiones_salida_Y):
    print(f"--- Output {i+1} ---")
    print("  R2 :", r2_score(Y_test1[:, i], Y_pred1[:, i]))
    aux['r2'].append(r2_score(Y_test1[:, i], Y_pred1[:, i]))
    print("MAE:", mean_absolute_error(Y_test1[:, i], Y_pred1[:, i]))
    aux['mae'].append(mean_absolute_error(Y_test1[:, i], Y_pred1[:, i]))
    print("  MSE:", mean_squared_error(Y_test1[:, i], Y_pred1[:, i]))
    aux['mse'].append(mean_squared_error(Y_test1[:, i], Y_pred1[:, i]))
    print("  RMSE:", np.sqrt(mean_squared_error(Y_test1[:, i], Y_pred1[:, i])))
    aux['rmse'].append(np.sqrt(mean_squared_error(Y_test1[:, i], Y_pred1[:, i])))
aux = pd.DataFrame(aux)

import matplotlib.pyplot as plt

for i in range(N_Dimensiones_salida_Y):
    plt.figure(figsize=(5,4))
    plt.scatter(Y_test1[:, i], Y_pred1[:, i], alpha=0.5)
    plt.plot([Y_test1[:, i].min(), Y_test1[:, i].max()],
             [Y_test1[:, i].min(), Y_test1[:, i].max()],
             'r--')
    plt.xlabel("Valor real")
    plt.ylabel("Predicción")
    plt.title(f"{dfy.keys()[i]} - Pred vs Real")
    plt.grid()
    plt.tight_layout()
    plt.show()
     """
mae_total1 = mean_absolute_error(Y_test1.flatten(), Y_pred1.flatten())
mse_total1 = mean_squared_error(Y_test1.flatten(), Y_pred1.flatten())
r2_total1 = r2_score(Y_test1, Y_pred1)

print("\n--- Total ---")
print("R² :", r2_total1)
print("MAE:", mae_total1)
print("MSE:", mse_total1)




5/5 [==============================] - 1s 8ms/step

--- Total ---
R² : 0.6382694788782493
MAE: 3.4484599857766316
MSE: 109.7411661882012


## CPU:0

In [30]:

# --- Red neuronal regresora multisalida ---
start_time = time()
early = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='loss', patience=5, factor=0.5)
with tf.device('/device:CPU:0'):
    # Definir el modelo
    model2 = Sequential(
        [
            InputLayer(input_shape=(N_Dimensiones_entrada_X,)),  # Capa de entrada con 8000 características
            BatchNormalization(),  # Normalización por lotes para mejorar la convergencia
            LeakyReLU(0.5),
            Dense(512, activation="leaky_relu", kernel_regularizer=l2(0.001)),  # Capa oculta con 1024 neuronas
            Dropout(0.2),  # Capa de abandono para evitar el sobreajuste
            LeakyReLU(0.25),
            Dense(128, activation="leaky_relu"),  # Otra capa oculta con 512 neuronas
            Dropout(0.1),  # Capa de abandono para evitar el sobreajuste
            Dense(16, activation="relu", kernel_regularizer=l2(0.001)),  # Otra capa oculta con 256 neuronas
            Dense(N_Dimensiones_salida_Y),  # Capa de salida con 8 valores (Target)
        ]
    )

    adam = Adam(learning_rate=0.001)
    # Compilar el modelo
    model2.compile(optimizer=adam , loss=["mse", "mse", "mse", "mse", "mse", "mse", "mse", "mse"], loss_weights=(0.1,1,1,1,1,1,1,1), metrics=["mae","mse"])  # Función de pérdida MSE y métricas MAE y MSE
stop_time = time()
print(f"Tiempo de compilacion del modelo 2: {stop_time - start_time:.2f} segundos")

Tiempo de compilacion del modelo 2: 0.34 segundos


In [31]:

# Resumen del modelo
model2.summary()
start_time = time()
# --- Entrenamiento ---
history2 = model2.fit(X_train, Y_train_scaled, epochs=500, batch_size=32, validation_data=(X_test, Y_test_scaled), verbose=1, callbacks=[early, reduce_lr])
stop_time = time()
print(f"Tiempo de entrenamiento del modelo 2: {stop_time - start_time:.2f} segundos")


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_4 (Batc  (None, 800)              3200      
 hNormalization)                                                 
                                                                 
 leaky_re_lu_8 (LeakyReLU)   (None, 800)               0         
                                                                 
 dense_16 (Dense)            (None, 512)               410112    
                                                                 
 dropout_8 (Dropout)         (None, 512)               0         
                                                                 
 leaky_re_lu_9 (LeakyReLU)   (None, 512)               0         
                                                                 
 dense_17 (Dense)            (None, 128)               65664     
                                                      

In [32]:



Y_pred_scaled2 = model2.predict(X_test)

# --- Evaluación ---
Y_pred2 = scaler_y.inverse_transform(Y_pred_scaled2)
Y_test2 = scaler_y.inverse_transform(Y_test_scaled)
aux = {}
aux['r2'] = []
aux['mae'] = []
aux['mse'] = []
aux['rmse'] = []

"""
# --- Evaluación ---
# Errores por cada salida
for i in range(N_Dimensiones_salida_Y):
    print(f"--- Output {i+1} ---")
    print("  R2 :", r2_score(Y_test2[:, i], Y_pred2[:, i]))
    aux['r2'].append(r2_score(Y_test2[:, i], Y_pred2[:, i]))
    print("MAE:", mean_absolute_error(Y_test2[:, i], Y_pred2[:, i]))
    aux['mae'].append(mean_absolute_error(Y_test2[:, i], Y_pred2[:, i]))
    print("  MSE:", mean_squared_error(Y_test2[:, i], Y_pred2[:, i]))
    aux['mse'].append(mean_squared_error(Y_test2[:, i], Y_pred2[:, i]))
    print("  RMSE:", np.sqrt(mean_squared_error(Y_test2[:, i], Y_pred2[:, i])))
    aux['rmse'].append(np.sqrt(mean_squared_error(Y_test2[:, i], Y_pred2[:, i])))
aux = pd.DataFrame(aux)

import matplotlib.pyplot as plt

for i in range(N_Dimensiones_salida_Y):
    plt.figure(figsize=(5,4))
    plt.scatter(Y_test2[:, i], Y_pred2[:, i], alpha=0.5)
    plt.plot([Y_test2[:, i].min(), Y_test2[:, i].max()],
            [Y_test2[:, i].min(), Y_test2[:, i].max()],
            'r--')
    plt.xlabel("Valor real")
    plt.ylabel("Predicción")
    plt.title(f"{dfy.keys()[i]} - Pred vs Real")
    plt.grid()
    plt.tight_layout()
    plt.show()
"""
mae_total2 = mean_absolute_error(Y_test2.flatten(), Y_pred2.flatten())
mse_total2 = mean_squared_error(Y_test2.flatten(), Y_pred2.flatten())
r2_total2 = r2_score(Y_test2, Y_pred2)

print("\n--- Total ---")
print("R² :", r2_total2)
print("MAE:", mae_total2)
print("MSE:", mse_total2)

5/5 [==============================] - 0s 6ms/step

--- Total ---
R² : 0.6076131129591852
MAE: 3.4916604885365126
MSE: 99.44790445458733
